# Build and test the stac container

This notebook is linked to https://eoap.github.io/mastering-app-package/containers/stac/

## Goal

Create a container image and run the stac step in a container.


## Setup the environment

In [ ]:
export WORKSPACE=/workspace/mastering-app-package
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

## Build the container

Inspect the container file:

In [ ]:
cat ${WORKSPACE}/water-bodies/command-line-tools/stac/Dockerfile

Build the container using `podman`:

In [ ]:
podman build --format docker -t localhost/stac:latest ${WORKSPACE}/water-bodies/command-line-tools/stac


Show the `stac` help:

In [ ]:
podman run --rm -it --env=PYTHONPATH=/app localhost/stac:latest python -m app --help

## Test the stac step in the container

Generate the STAC Catalog with detected water bodies:

In [ ]:
podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/workspace/mastering-app-package/runs,target=/runs \
    --mount=type=bind,source=/workspace/mastering-app-package/runs/otsu.tif,target=/inputs/otsu.tif,readonly \
    --workdir=/runs \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --env=HOME=/runs \
    --env=PYTHONPATH=/app \
    localhost/stac:latest \
    python \
    -m \
    app \
    --input-item \
    https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A \
    --water-body \
    /inputs/otsu.tif

List the outputs:

In [ ]:
tree ${RUNTIME}